# ATTACHMENT 2: Financial Analysis of Booking Holdings Inc. (BKNG)



## Preliminary Code
First of all, we import relevant libraries and extract the company's financial data useful for the subsequent analysis.

In [ ]:
# Loading useful libraries
import yfinance as yf
import pandas as pd
import numpy as np

# Extracting historical stock prices thorugh Yahoo Finance
ticker = 'BKNG' # Booking Holdings Inc.
stock_data = yf.download(ticker, start='1997-03-01', end='2024-04-01')
stock_5years = yf.download(ticker, start='2019-04-01', end='2024-04-01')

# Extracting the company's general financial data
company = yf.Ticker(ticker)
balance_sheet = company.balance_sheet
financials = company.financials
#tax_rate = financials['2023-12-31'].loc['Tax Rate For Calcs'] (We used this initially and then changed it to the US Corporate tax rate)
tax_rate = .21
cashflow = company.cashflow
dividends = company.dividends
dividend = 35 # Assuming a constant dividend of 35 USD per share: no dividend data found on YFinance
market_cap = company.info['marketCap']
enterprise_value = company.info['enterpriseValue']
revenue = financials.loc['Total Revenue'].iloc[0]
operating_income = financials.loc['Operating Income'].iloc[0]
capex = cashflow.loc['Capital Expenditure'].iloc[0]
nwc = balance_sheet.loc['Current Assets'] - balance_sheet.loc['Current Liabilities']

# Visualizing stock data format
print("\nStock data (last 5 rows):\n", stock_data.tail())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Stock data (last 5 rows):
                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2024-03-22  3624.969971  3646.120117  3604.459961  3624.729980  3624.729980   
2024-03-25  3608.439941  3629.830078  3569.500000  3626.939941  3626.939941   
2024-03-26  3649.000000  3688.909912  3640.600098  3661.080078  3661.080078   
2024-03-27  3677.939941  3711.429932  3656.139893  3673.500000  3673.500000   
2024-03-28  3690.000000  3694.639893  3618.399902  3627.879883  3627.879883   

            Volume  
Date                
2024-03-22  205400  
2024-03-25  166600  
2024-03-26  274900  
2024-03-27  209200  
2024-03-28  274100  


## Cost of Equity Estimation
We compute the Cost of Equity with two different methods: as the sum of Dividend Growth Rate and Dividend Yield and using the CAPM formula.

In [ ]:
# Estimating the cost of equity based on the dividend growth and yield
div_growth = 0  # Since a single dividend payment exists
div_yield = dividends.mean() / stock_data['Close'].mean()
cost_of_equity = div_growth + div_yield # Note: in this case the cost of equity is equal to the dividend yield, since the dividend has growth = 0
print(f"\n\nEstimated Dividend Growth Rate = {div_growth:.2%}")
print(f"Dividends-estimated Cost of Equity = {cost_of_equity:.2%}")

# Extracting S&P 500 index's data
historical_data_SaP = yf.download('SPY', start='2014-03-01', end='2024-04-01')

# Estimating our own beta to be used in the CAPM
market_prices2 = yf.download('SPY', start='2019-04-01', end='2024-04-01')['Adj Close']

# Extracting the risk-free rate based on the 10-year Treasury Note
treasury = yf.Ticker("^TNX")
risk_free_rate = treasury.history(period="5y")['Close'].iloc[-1] / 100  # Converting basis points to decimal
print(f"\nEstimated Risk-Free Rate = {risk_free_rate:.2%}")

# Getting historical returns for the S&P 500
historical_data = historical_data_SaP['Adj Close']
historical_data = historical_data.dropna()
market_return = np.log(historical_data/historical_data.shift(1))
market_return.dropna(inplace = True)
annual_return = market_return.mean()*252
print(f"Estimated Annual Market Return = {annual_return:.2%}")

# Extracting the daily returns
stock_returns2 = stock_5years['Adj Close'].pct_change().dropna()
market_returns2 = market_prices2.pct_change().dropna()

# Calculating beta as stock-market return covariance divided by market returns' variance
covariance = np.cov(stock_returns2, market_returns2)[0][1]
market_variance = np.var(market_returns2)
beta2 = covariance / market_variance

# Estimating the cost of equity using the CAPM formula
cost_of_equity2 = risk_free_rate + beta2 * (annual_return - risk_free_rate)
print(f"CAPM-estimated Cost of Equity = {cost_of_equity2:.2%}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Estimated Dividend Growth Rate = 0.00%
Dividends-estimated Cost of Equity = 0.97%

Estimated Risk-Free Rate = 4.71%
Estimated Annual Market Return = 12.16%
CAPM-estimated Cost of Equity = 13.38%


We obviously obtain two different values; the dividend-estimated Cost of Equity in particular will be used for the DCF Perpetuity analysis of the Stock Price.

## P/E and P/B Calculation

We estimate the Price-to-Earnings and Price-to-Book Ratios considering data from the company's IPO, then we compare the results with the values extracted - where possible - via Yahoo Finance and with the performance indicators of some BKNG's market competitors; this analysis might be useful in determining the Stock Price's fairness.

In [ ]:
# Computing the price-to-earnings ratio (per-share values considered)
net_income = financials.loc['Net Income'].iloc[0]
earning_per_share = net_income / balance_sheet.loc['Share Issued'].iloc[0]
price_per_share = stock_data['Close']
P_E_ratio = price_per_share / earning_per_share
print(f"Estimated P/E Ratio = {P_E_ratio.iloc[-1]:.3f}")

# Extracting the same ratio from Yahoo Finance for comparison
print(f"Yahoo-Finance P/E Ratio = {company.info['trailingPE']:.3f}")

# Computing the market-price to book-value ratio
book_value_ps = balance_sheet.loc['Total Equity Gross Minority Interest'].iloc[0] / balance_sheet.loc['Share Issued'].iloc[0]
P_B_ratio = price_per_share / book_value_ps
print(f"\nEstimated P/B Ratio = {P_B_ratio.iloc[-1]:.3f}")

# Extracting the same ratio from Yahoo Finance for comparison
# Note: would not work as 'priceToBook' is not found, since 'bookValue' is negative

Estimated P/E Ratio = 54.175
Yahoo-Finance P/E Ratio = 29.811

Estimated P/B Ratio = -84.679


### Comparative Analysis of Ratios

MSN Money already did an analysis on Booking Holdings' Price-over-Earnings and Price-to-Book, reporting end-December results (P/E: 30.07, P/B: -45.51) somewhat dissimilar from ours. The discrepancy must be due to the long-period data aggregation we operated, evidently not intended to represent the up-to-date situation. Similarly, the difference between our P/E and Yahoo Finance's one is explainable by the fact that the latter is a 12-month trailing value.

Moreover, compared to the competitors identified by MSN Money (i.e., META, AAPL-Apple and AMZN-Amazon), we notice that BKNG presented quite controversial indicators for F.Y. 2023: the focus firm had the second highest P/E score - of 30.21, compared to AMZN's record one of 52.40 - but the only negative P/B too - of -47.22. We may also consider as a Booking's market rival ABNB-Airbnb; the latter firm has totally different ratios (i.e., P/E: 18.81, P/B: 11.04) indicating a more balanced situation and a positive Equity amount.

(Source: https://www.msn.com/en-us/money/stockdetails/fi-az6g8m?id=az6g8m)

### Negative Equity Explaination

It is important to notice that BKNG has a negative Total Equity reported for 2023, precisely of -2,744B\$. This situation is allowed in US financial system: by regulation a negative Equity does not necessarily imply bankruptcy, since the firm is still entrusted to meet its obligations if demonstrating sufficient Cash Flows. This provision is coherent with the American approach of encouragement towards entrepreneurship and growth, somehow limiting creditors' safety.

On the other hand, a situation similar to BKNG's one would not be verifiable in Italy, as the Corporate Law requires LLCs (i.e., S.r.l- and S.p.A-type firms) to maintain a minimum amount of Equity. Any company missing to satisfy this requirement might be restored through financial measures (e.g., capital injection) or potentially face liquidation proceedings. In fact, the Italian Civil Code - traditionally emphasizing  creditor protection - aims to maintain financial stability even by forcing firms to take corrective actions rather than allowing financial distress indicated by negative Equity.

## Stock Price Estimation
We estimate the Stock Price by two methods:

1) Projected Free Cash Flows system;

2) Discounted Cash Flow Perpetuity model.

In [ ]:
# 1) Projected Free Cash Flows

# Calculating financial metrics
revenue = financials.loc['Total Revenue'].iloc[0]
operating_income = financials.loc['Operating Income'].iloc[0]
capex = cashflow.loc['Capital Expenditure'].iloc[0]
nwc = balance_sheet.loc['Current Assets'] - balance_sheet.loc['Current Liabilities']
operating_margin = (operating_income / revenue) * 100
capex_percentage_of_revenue = (capex / revenue) * 100
change_in_nwc_percentage_of_revenue = ((nwc.iloc[0] - nwc.iloc[-1]) / revenue) * 100
print("Operating Margin:", round(operating_margin, 2), "%")
print("Capex Percentage of Revenue:", round(capex_percentage_of_revenue, 2), "%")
print("Change in NWC Percentage of Revenue:", round(change_in_nwc_percentage_of_revenue, 2), "%")

# Calculating the revenue growth and prediciting the future revenue (with gradual decrease of g to 6%)
revenue_growth = financials.loc['Total Revenue'].iloc[::-1].pct_change()
revenue_growth.dropna(inplace=True)
last_known_growth = revenue_growth.iloc[-1]
future_years = 6  # Number of future years to predict
final_growth_rate = 0.06  # Target final growth rate

# Creating a linear interpolation between the last known growth and the final target growth
predicted_growth = np.linspace(last_known_growth, final_growth_rate, future_years)

# Projecting future revenues
current_revenue = financials.loc['Total Revenue'].iloc[0]  # Revenue current year
projected_revenues = []
for i in range(0,6):
    future_revenue = current_revenue * ((1 + predicted_growth[i]) ** i)
    projected_revenues.append(future_revenue)

# Projecting future free cash flows
free_cash_flows = []
for revenue in projected_revenues:
    operating_income = revenue * operating_margin
    capex = revenue * capex_percentage_of_revenue
    change_in_nwc = revenue * change_in_nwc_percentage_of_revenue
    free_cash_flow = operating_income + capex + change_in_nwc
    free_cash_flows.append(free_cash_flow)

# Estimating the stock price
weighted_avg_cost_of_capital = 0.12 # Obtained from future calculations for the WACC
debt = balance_sheet.loc['Long Term Debt'].iloc[0]
present_value_flows = sum(cf / ((1 + weighted_avg_cost_of_capital) ** (i+1)) for i, cf in enumerate(free_cash_flows))
shares_outstanding = balance_sheet.loc['Share Issued'].iloc[0]  # Assuming outstanding shares
stock_price = (present_value_flows - debt) / shares_outstanding
stock_price = stock_price.mean()
print(f"Future-CFs-estimated Stock Price = {stock_price:.2f}$")

Operating Margin: 27.31 %
Capex Percentage of Revenue: -1.61 %
Change in NWC Percentage of Revenue: -23.76 %
Future-CFs-estimated Stock Price = 3210.61$


Using the projected future Free Cash Flows system, we build a Stock Price estimation which highly relies on the future Revenue Growth assessment. It is also important to notice that we assumed the WACC equal to the one we estimated (dedicated code section below).

The obtained Stock Price (3210.61\$) is a bit lower than the actual one; this could be due to disparate factors, but the relative error is acceptably low.

In [ ]:
# 2) Discounted Cash Flow Perpetuity

# Computing the present value of the perpetuity of stock using the DCF
pvperp = dividend / cost_of_equity
print(f"DCF-Perpetuity-estimated Stock Price = {pvperp:.2f}$")

DCF-Perpetuity-estimated Stock Price = 3596.44$


However, the Stock Price computed via the DCF perpetuity model - with Cost of Equity = 0.97% and no Dividend Growth Rate - returns a value (3596.44\$) very similar to Yahoo Finance's one (i.e., 3568.87\$) as of 01/04/2024 - end date of the data used in our analysis, which indicates a rather fair Stock Price.

## ROE and Plowback Estimation

In [ ]:
# Computing the ROE and plowback
roe = net_income / balance_sheet.loc['Total Equity Gross Minority Interest'].iloc[0]
print(f"Estimated Return On Equity = {roe:.2f}")
retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0]
total_revenue = financials.loc['Total Revenue'].iloc[0]
plowback = retained_earnings / total_revenue
payout = 1 - plowback
print(f"Estimated Plowback Ratio = {plowback:.2f}")

Estimated Return On Equity = -1.56
Estimated Plowback Ratio = 1.49


## Beta Computation
We compute the Beta manually and then extract it via Yahoo Finance; conscious  that the latter value represents the monthly 5-year Beta.

In [ ]:
# Extracting market's and stock's daily returns
stock_5years = yf.download(ticker, start='2019-04-01', end='2024-04-01')
market_prices = yf.download('SPY', start='2019-04-01', end='2024-04-01')['Adj Close']
stock_returns = stock_5years['Adj Close'].pct_change().dropna()
market_returns = market_prices.pct_change().dropna()

# Calculating beta as stock-market return covariance divided by market returns' variance
covariance = np.cov(stock_returns, market_returns)[0][1]
market_variance = np.var(market_returns)
beta = covariance / market_variance
print(f"\n\nEstimated Beta = {beta:.3f}")

# Extracting the beta from YFinance for comparison
beta_yahoo = company.info['beta']
print("Yahoo-Finance Beta =" ,beta_yahoo)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Estimated Beta = 1.165
Yahoo-Finance Beta = 1.402


The computed Beta (1.165) indicates that the company has a slightly higher Volatility compared to the overall market: if the market goes up or down, the Stock Price is expected to go up or down as well, but to a slightly higher degree than the market itself. This suggests that the firm has a risk profile slightly above the market average, making it a somewhat riskier investment than the market as a whole.

## Cost of Capital Assessment

We compute the after-tax Weighted Average Cost of Capital: firstly we calculate the Cost of Debt, then the WACC itself using the Cost of Debt and the CAPM-estimated Cost of Equity.

In [ ]:
# Calculating the cost of debt
interest_expense = financials.loc['Interest Expense'].iloc[0]
total_debt = balance_sheet.loc['Total Debt'].iloc[0]
cost_of_debt = interest_expense / total_debt
print(f"Estimated Cost of Debt = {cost_of_debt:.2%}")

# Estimating the WACC after tax
total_debt = enterprise_value - market_cap
E_V = market_cap / enterprise_value
D_V = total_debt / enterprise_value
wacc = (E_V * cost_of_equity2) + (D_V * cost_of_debt * (1 - tax_rate))
print(f"Estimated WACC = {wacc:.2%}")

Estimated Cost of Debt = 6.07%
Estimated WACC = 13.15%


In order to assess their fairness, we compare our calculations to those of GuruFocus (updated to the end of April 2024). Firstly, our CAPM-estimated Cost of Equity (13.39%) is very similar to the aforementioned website's one (i.e., 12.30%), also computed via the CAPM formula. An even lower discrepancy is found comparing the Cost of Debt estimations, as GuruFocus' value is of 6.46% and ours of 6.07%. This led the website's computations to a WACC of 11.55%, which is slightly lower than ours (13.18%), possibly also due to the CAPM-estimated Cost of Equity.

(Source: https://www.gurufocus.com/term/wacc/BKNG#:~:text=Booking%20Holdings%20(NAS%3ABKNG)%20WACC%20%25&text=As%20of%20today%20(2024-04,of%20capital%20is%2011.44%25%25))